<a href="https://colab.research.google.com/github/AjMing/BigData/blob/main/Demo_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1e6c07c9c52ff76c297884730ed5f0e26436694e760f85a6c92ffb449699b864
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import findspark
findspark.init()

In [ ]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context.
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Creating a spark context class
sc = SparkContext()

In [ ]:
spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
data=[('A',1),('B',2),('C',3)]

In [ ]:
rdd=sc.parallelize(data)

In [ ]:
print(rdd.glom().collect())

[[('A', 1)], [('B', 2), ('C', 3)]]


# Create DataFrame

In [ ]:
dfFromRDD=rdd.toDF()
dfFromRDD.printSchema()
dfFromRDD.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+---+---+
| _1| _2|
+---+---+
|  A|  1|
|  B|  2|
|  C|  3|
+---+---+



In [ ]:
header=['key','value']

In [ ]:
dfFromRDD1=rdd.toDF(header)
dfFromRDD1.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [ ]:
dfFromRDD.show()

+---+---+
| _1| _2|
+---+---+
|  A|  1|
|  B|  2|
|  C|  3|
+---+---+



In [ ]:
dfFromRDD1.show()

+----+-----+
|word|count|
+----+-----+
|   A|    1|
|   B|    2|
|   C|    3|
+----+-----+



Option2: Import RDD with column

In [ ]:
dfFromRDD2=spark.createDataFrame(rdd).toDF(*header)

In [ ]:
dfFromRDD2.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [ ]:
dfFromRDD2.show()

+----+-----+
|word|count|
+----+-----+
|   A|    1|
|   B|    2|
|   C|    3|
+----+-----+



Option3: Import the data directly

In [ ]:
dfFromRDD3=spark.createDataFrame(data).toDF(*header)
dfFromRDD3.printSchema()
dfFromRDD3.show()

root
 |-- key: string (nullable = true)
 |-- value: long (nullable = true)

+---+-----+
|key|value|
+---+-----+
|  A|    1|
|  B|    2|
|  C|    3|
+---+-----+



In [ ]:
sc.stop()

# Import DataFrame

In [ ]:
# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames Import example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
#dataframe native

In [ ]:
dataframe = spark.read.csv(path+"students.csv",header='true',inferSchema=True)
dataframe.show()
dataframe.printSchema()


+------+---+----+
|  Name|AGE| GPA|
+------+---+----+
|Edward| 15|NULL|
|   Ken| 19| 2.5|
|  Ploy| 20| 3.8|
|   Que| 25| 2.9|
|   NUT| 24| 3.9|
+------+---+----+

root
 |-- Name: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- GPA: double (nullable = true)



In [ ]:
#Get Column Name
dataframe.columns

['Name', 'AGE', 'GPA']

In [ ]:
df_new=dataframe.withColumn('Graduation age',dataframe['AGE']+4)


In [ ]:
df_new.show()

+------+---+----+--------------+
|  Name|AGE| GPA|Graduation age|
+------+---+----+--------------+
|Edward| 15|NULL|            19|
|   Ken| 19| 2.5|            23|
|  Ploy| 20| 3.8|            24|
|   Que| 25| 2.9|            29|
|   NUT| 24| 3.9|            28|
+------+---+----+--------------+



In [ ]:
from pyspark.sql.functions import avg
df_new.select(avg(df_new["GPA"])).show()


+--------+
|avg(GPA)|
+--------+
|   3.275|
+--------+



In [ ]:
df_new.filter(df_new["Graduation age"] > 21).show()

+----+---+---+--------------+
|Name|AGE|GPA|Graduation age|
+----+---+---+--------------+
| Ken| 19|2.5|            23|
|Ploy| 20|3.8|            24|
| Que| 25|2.9|            29|
| NUT| 24|3.9|            28|
+----+---+---+--------------+



In [ ]:
df=df_new.na.drop()
df.show()

+----+---+---+--------------+
|Name|AGE|GPA|Graduation age|
+----+---+---+--------------+
| Ken| 19|2.5|            23|
|Ploy| 20|3.8|            24|
| Que| 25|2.9|            29|
| NUT| 24|3.9|            28|
+----+---+---+--------------+



In [ ]:
from pyspark.sql.functions import udf


In [ ]:
def UpperCase(str):
  return str.upper()

In [ ]:
upperCaseUDF=udf(lambda z: UpperCase(z))

In [ ]:
df.withColumn("Uppercase Name",upperCaseUDF("Name")).show()

+----+---+---+--------------+--------------+
|Name|AGE|GPA|Graduation age|Uppercase Name|
+----+---+---+--------------+--------------+
| Ken| 19|2.5|            23|           KEN|
|Ploy| 20|3.8|            24|          PLOY|
| Que| 25|2.9|            29|           QUE|
| NUT| 24|3.9|            28|           NUT|
+----+---+---+--------------+--------------+



In [ ]:
# Read the dataset into a spark dataframe using the `read.json()` function
df = spark.read.json(path+"people.json").cache()

In [ ]:
# Print the dataframe as well as the data schema
df.show()


+----+----+---------+
| age| gpa|     name|
+----+----+---------+
|null|null|  Krisada|
|  30|null|   Pradya|
|  19|null|   Justin|
|null| 3.9|Teeravach|
+----+----+---------+



In [ ]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- gpa: double (nullable = true)
 |-- name: string (nullable = true)



**Task 2**:** Explore the data using DataFrame** functions and SparkSQL




In this section, we explore the datasets using functions both from dataframes as well as corresponding SQL queries using sparksql. Note the different ways to achieve the same task!# New Section

In [ ]:
# Select and show basic data columns



+----+
| age|
+----+
|null|
|  30|
|  19|
|null|
+----+

+---------+
|     name|
+---------+
|  Krisada|
|   Pradya|
|   Justin|
|Teeravach|
+---------+



In [ ]:
# Register the DataFrame as a SQL temporary view


In [ ]:
spark.sql("SELECT name,age FROM people").show()

+---------+----+
|     name| age|
+---------+----+
|  Krisada|null|
|   Pradya|  30|
|   Justin|  19|
|Teeravach|null|
+---------+----+



In [ ]:
df.sort("age").show()
spark.sql("SELECT * FROM people order by age desc").show()

+----+----+---------+
| age| gpa|     name|
+----+----+---------+
|null|null|  Krisada|
|null| 3.9|Teeravach|
|  19|null|   Justin|
|  30|null|   Pradya|
+----+----+---------+

+----+----+---------+
| age| gpa|     name|
+----+----+---------+
|  30|null|   Pradya|
|  19|null|   Justin|
|null|null|  Krisada|
|null| 3.9|Teeravach|
+----+----+---------+



In [ ]:
df.withColumnRenamed('name','Students_name').show()

+----+----+-------------+
| age| gpa|Students_name|
+----+----+-------------+
|null|null|      Krisada|
|  30|null|       Pradya|
|  19|null|       Justin|
|null| 3.9|    Teeravach|
+----+----+-------------+



In [ ]:
df.withColumnRenamed('Students_name','name').show()

+----+----+---------+
| age| gpa|     name|
+----+----+---------+
|null|null|  Krisada|
|  30|null|   Pradya|
|  19|null|   Justin|
|null| 3.9|Teeravach|
+----+----+---------+



In [ ]:
df_new=df.withColumn('Graduation age',df['age']+1)
df_new.show()


+----+----+---------+--------------+
| age| gpa|     name|Graduation age|
+----+----+---------+--------------+
|null|null|  Krisada|          null|
|  30|null|   Pradya|            31|
|  19|null|   Justin|            20|
|null| 3.9|Teeravach|          null|
+----+----+---------+--------------+



In [ ]:
df_new.drop('Graduation age').show()

+----+----+---------+
| age| gpa|     name|
+----+----+---------+
|null|null|  Krisada|
|  30|null|   Pradya|
|  19|null|   Justin|
|null| 3.9|Teeravach|
+----+----+---------+



In [ ]:
# Perform basic filtering

df.filter(df["age"] > 21).show()
spark.sql("SELECT * FROM people WHERE age > 21").show()

+---+----+------+
|age| gpa|  name|
+---+----+------+
| 30|null|Pradya|
+---+----+------+

+---+----+------+
|age| gpa|  name|
+---+----+------+
| 30|null|Pradya|
+---+----+------+



In [ ]:
# Perfom basic aggregation of data

df.groupBy("age").count().show()
spark.sql("SELECT age, COUNT(age) as count FROM people GROUP BY age").show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    2|
|  30|    1|
+----+-----+

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    0|
|  30|    1|
+----+-----+



# Exercise 1



 **Exercise1** read the people2.json file into the notebook, load it into a dataframe and apply SQL operations to determine the average age in our people2 file.

In [ ]:
#Exercise 1
# df = spark.read...
df1 = spark.read.json(path+"people2.json")
# df.createTempView..
# spark.sql("SELECT ...")
from pyspark.sql import functions as f
df1.createOrReplaceTempView("student")


**Exercise2** read the people2.json file into the notebook, load it into a dataframe and apply SQL operations to determine NUMBER OF PEOPLE IN EACH PROGRAM/ AVERAGE  IN EACH PROGRAM


In [ ]:
df1.groupby('major').agg(f.count('*'),f.avg('age')).show()

+------+--------+--------+
| major|count(1)|avg(age)|
+------+--------+--------+
|EGCO/M|       5|   24.75|
|  EGCO|       2|    22.0|
+------+--------+--------+



In [ ]:
from pyspark.sql.functions import mean
df1.na.fill(df1.select(f.mean(df['age'])).collect()[0][0],['age'])

df1.show()

+------+----+-----------+
| Major| age|       name|
+------+----+-----------+
|EGCO/M|  30|  Nartdanai|
|EGCO/M|  29|      Tanut|
|EGCO/M|null|  Nattapark|
|EGCO/M|  20|  Weerawich|
|  EGCO|  22|Threerapong|
|EGCO/M|  20|  Weerawich|
|  EGCO|  22|Threerapong|
+------+----+-----------+



In [ ]:
merge.show()

+----+---+---+-----+---+
|name|age|gpa|Major|age|
+----+---+---+-----+---+
+----+---+---+-----+---+



In [ ]:
def UpperCase(str):
  return str.upper()

In [ ]:
from pyspark.sql.functions import udf

upperCaseUDF=udf(lambda z: UpperCase(z))

In [ ]:
df2=df1.withColumn("Uppercase Name", upperCaseUDF("name"))

In [ ]:
df2.show()

+------+----+-----------+--------------+
| Major| age|       name|Uppercase Name|
+------+----+-----------+--------------+
|EGCO/M|  30|  Nartdanai|     NARTDANAI|
|EGCO/M|  29|      Tanut|         TANUT|
|EGCO/M|null|  Nattapark|     NATTAPARK|
|EGCO/M|  20|  Weerawich|     WEERAWICH|
|  EGCO|  22|Threerapong|   THREERAPONG|
|EGCO/M|  20|  Weerawich|     WEERAWICH|
|  EGCO|  22|Threerapong|   THREERAPONG|
+------+----+-----------+--------------+



**Exercise3** Remove Duplicate

In [ ]:
#Exercise 3

**Exercise4** REMOVE NULL VALUE or use average value to fill in the NULL

In [ ]:
#Exercise 4

In [ ]:
#close session
spark.stop()